In [1]:
import numpy as np
import pickle
from time import time
from joblib import Parallel, delayed
import sys
from src.models.mlp import MLP
from src.eval import RydbergEvaluator
from src.eval.eval_rydberg import est_density_from_z_measurements,determine_phase_1D,est_order_param_1D,phase2img,est_phase_diagram,est_order_param_1D_fourier_from_measurements,est_order_param_1D_fourier,est_order_param_1D_from_measurements
from src.data.loading.dataset_rydberg import RydbergDataset,unif_sample_on_grid
# Transformer
import argparse
from constants import *
from src.training.rydberg_trainers import RydbergConditionalTransformerTrainer
from src.models.transformer import init_conditional_transformer
from src.models.mlp import MLP


import torch
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from tqdm.notebook import tqdm,trange
import os

from src.utils import plot_phase_diagram

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
%matplotlib inline


2025-09-30 16:48:04.795813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759243685.002110   91989 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759243685.125760   91989 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759243686.160695   91989 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759243686.160744   91989 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759243686.160750   91989 computation_placer.cc:177] computation placer alr

In [2]:
# Here we define a base schedule -- 
base_time = 3.5
ts = np.array([0,0.2,base_time])
omegas = np.array([0, 5, 5])
deltas = np.array([-10, -10, 15])
total_time = 15 # Total adiabatic evolution time of the Bloqade simulation
# We propotionally lengthen the schedules for Omega and Delta by time_ratio = total_time/base_time
time_ratio = total_time/base_time

In [3]:
n_qubits = 31 # number of Rydberg atoms in the 1D lattice

dim=1 # dimension of the system
ny = 1 # since we are working in 1D, this variable is fixed to 1
nx = n_qubits # effectively, we are working on a 2D lattice of dimensions nx*ny, where nx=n_qubits and ny=1.
z2_threshold=0.7 # threshold for the Z2 order parameter to determine a state is in Z2 phase
z3_threshold = 0.6 # threshold for the Z3 order parameter to determine a state is in Z3 phase

# We load simulation data for the lattice defined above with the adiabatic evolution scheduler preset above.
folder = f'data/1D-Phase_{nx}/1D-Phase_{nx}/{float(np.round(total_time,2))}µs/'
# folder

In [4]:
# extra variables we want the conditional generative variable to condition on, except for "nx", "ny", "interaction_range".
# detuning = Delta/Omega
extra_variables = ["detuning",] 
meta_dataset = RydbergDataset(dim=dim,nx = nx, ny=ny, folder=folder,n_threads=1, 
                                         var_name='interaction_range',variables = extra_variables) 
meta_dataset.est_order_params()
meta_dataset.info["phase" ] = determine_phase_1D(meta_dataset.info["Z2"], meta_dataset.info["Z3"],z2_threshold=z2_threshold,
                                                z3_threshold=z3_threshold
                                            )

In [5]:
sns.set_style('white')
hue_order = ['Disordered','Z2','Z3']
plot_df = meta_dataset.info.copy()
plot_df = plot_df.loc[(plot_df['detuning'] >=-1) & (plot_df['interaction_range'] <= 2.8) & (plot_df['interaction_range'] > 1)]
# fig = plot_phase_diagram(plot_df,title=f"1D Lattice of {n_qubits} Rydberg Atoms: Phase Diagram",hue_order = hue_order,
#                         legend=True)

In [6]:
def prepare_train_set(meta_dataset, df=None, n_measurements:int = -1, x_bins=10,y_bins=10):
    train_set = {}
    if df is None: df = meta_dataset.info
    train_idxes, train_df = unif_sample_on_grid(df.copy(),x_bins=x_bins,y_bins=y_bins)#,x_range=(0.4,1.78),y_range=(1.4,2.4))
    # train_idxes = plot_df.index.values
    train_keys = meta_dataset.keys[train_idxes]
    train_set.update(meta_dataset.prepare_train_set(train_keys,n_measurements=n_measurements))
    return train_set, train_idxes
    

In [7]:
# Decide to load a pretrained model or train from scratch
load_pretrained = True

In [8]:
if load_pretrained:
    # If loading a pretrained model, we need to also load the training set that 
    # it was trained on (specified by sampled indices of the meta_dataset.info DataFrame)
    train_idxes = np.load('logs/rydberg_1D/train_idxes.npy')
    train_set = pickle.load(open('logs/rydberg_1D/train_set.pkl','rb'))
else:
    # If train from scratch, we sample training data from the phase diagram specified by plot_df
    train_set, train_idxes = prepare_train_set(meta_dataset,df=plot_df)

In [9]:
import random

# This will consistently give you the same ordering
train_set_keys = list(train_set.keys())
train_set_values = list(train_set.values())

# Select N random indices
N = 20
selected_indices = random.sample(range(len(train_set_keys)), N)

# These will correspond to the same positions
selected_dict_items = {train_set_keys[i]: train_set[train_set_keys[i]] for i in selected_indices}
selected_array_items = [train_idxes[i] for i in selected_indices]

train_set = selected_dict_items
train_idxes = selected_array_items

In [12]:
# If you want to save this training set, you can un-comment the following lines:
# Create all necessary directories
output_dir = f'logs/rydberg_1D/{N}'
os.makedirs(output_dir, exist_ok=True)  # exist_ok=True prevents error if dir exists


np.save(f'logs/rydberg_1D/{N}/train_idxes.npy',train_idxes)
pickle.dump(train_set, open(f'logs/rydberg_1D/{N}/train_set.pkl','wb'))

## Train a Conditional Generative Model

We first define some hyperparameters as follows

In [13]:
def parse_args(args=[]):
    parser = argparse.ArgumentParser()
    parser.add_argument('--data-dir', type=str, default='logs/rydberg/debug/')
    parser.add_argument('--dim',type=int,default=1)
    parser.add_argument('--nx',type=int,default=19)
    parser.add_argument('--ny',type=int,default=1)
    parser.add_argument('--total_time',type=float,default=6)
    parser.add_argument('--tf-arch', type=str, default='transformer_l4_d128_h4')
    parser.add_argument('--train-id', type=str, default="debug")
    parser.add_argument('--reps', type=int, default=1)
    parser.add_argument('--ns', type=int, default=800, help='number of samples per hamiltonian')
    parser.add_argument('--iterations', type=int, default=50000, help="training iterations")
    parser.add_argument('--eval-every', type=int, default=100)
    parser.add_argument('--eval-samples', type=int, default=10000, help='number of generated samples for evaluation')
    parser.add_argument('--k', type=int, default=1, help='number of buckets for median of means estimation')
    parser.add_argument('--n_cpu', type=int, default=8, help='number of cpu threads to use during batch generation')
    parser.add_argument('--verbose', type=int, default=1, choices=[0, 1])
    parser.add_argument('--epoch-mode', type=int, default=1, choices=[0, 1])
    parser.add_argument('--condition-mode', type=int, default=0, choices=[0, 1])
    parser.add_argument('--seed', type=int, default=None)
    return parser.parse_args(args)

def get_hyperparams(**kwargs):
    hparams = argparse.Namespace(
        lr=1e-3,
        wd=0,
        bs=512,
        dropout=0.0,
        lr_scheduler=WARMUP_COSINE_SCHEDULER,
        warmup_frac=0.,
        final_lr=1e-7,
        smoothing=0.0,
        use_padding=0,
        val_frac=0.25,
        cattn=0
    )

    for k, v in kwargs.items():
        setattr(hparams, k, v)

    return hparams
args = parse_args()
hparams = get_hyperparams()

### Select Device (GPU/CPU)
`gpu_idx < 0`: Use CPU

`gpu_idx > 0`: Use NVIDIA GPU

In [14]:
gpu_idx = 2
device = torch.device(f"cuda:{gpu_idx}") if gpu_idx >= 0 else torch.device('cpu')

print(f"Device: {device}")
print(f"CUDA available: {torch.cuda.is_available()}")


Device: cuda:2
CUDA available: False


In [15]:
num_outcomes = 2 # for rydberg systems
n_vars = len(list(train_set.keys())[0])


rng = np.random.default_rng(seed=args.seed)
# setup transformer
d_model = TF_ARCHS[args.tf_arch]['d_model']
n_head = TF_ARCHS[args.tf_arch]['n_head']
n_layers = TF_ARCHS[args.tf_arch]['n_layers']
assert d_model % n_head == 0, 'd_model must be integer multiple of n_head!'

We construct a MLP (fully-connected net) as the encoder and a transformer as the generative model, and then train the model

In [16]:
encoder = MLP(input_size=n_vars, output_size=d_model, 
              n_layers=1, hidden_size=128, activation='ELU', 
              input_layer_norm=False,
              output_batch_size=None, device=device,
             output_factor=1.)

transformer = init_conditional_transformer(
        n_outcomes=num_outcomes,
        encoder=encoder,
        n_layers=n_layers,
        d_model=d_model,
        d_ff=4 * d_model,
        n_heads=n_head,
        dropout=hparams.dropout,
        version=hparams.use_padding,
        use_prompt=False, #***
)

AssertionError: Torch not compiled with CUDA enabled

In [17]:
trainer = RydbergConditionalTransformerTrainer(model=transformer,
                                  train_dataset=train_set,
                                  test_dataset=None,
                                  iterations=args.iterations,
                                  lr=hparams.lr,
                                  final_lr=hparams.final_lr,
                                  lr_scheduler=hparams.lr_scheduler,
                                  warmup_frac=hparams.warmup_frac,
                                  weight_decay=hparams.wd,
                                  batch_size=hparams.bs,
                                  rng=rng,
                                  smoothing=hparams.smoothing,
                                  eval_every=args.eval_every,
                                  transfomer_version=hparams.use_padding,
                                  device=device)
model_name = f'transformer_nq-{n_qubits}_iter-{args.iterations//1000}k'
print('Training iterations:', args.iterations)

NameError: name 'transformer' is not defined

In [ ]:
trainer.train()

# Create all necessary directories
output_dir = f'logs/rydberg_1D/{N}'
os.makedirs(output_dir, exist_ok=True)  # exist_ok=True prevents error if dir exists

# Now save the model
model_path = f'{output_dir}/{model_name}.pth'
torch.save(transformer, model_path)
print(f"Model saved to: {model_path}")

# torch.save(transformer,f'logs/rydberg_1D/{N}/{model_name}.pth') # You can save the trained model